In [1]:
import json
import mysql.connector

In [2]:
# Connection to the MySQL database
connection = mysql.connector.connect(
  host = '127.0.0.1',
  user = 'root',
  password = 'password',
  database = 'esuper'
)

# Create a cursor to execute SQL queries
cursor = connection.cursor()

In [3]:
# Products Insertion

# Read the JSON file
with open("./Data/products.json", encoding="utf-8" ) as file:
  products_json = json.load(file)

# create the insertion query
insert_products_query = "INSERT INTO product (product_id, product_name, category, subcategory, img) VALUES (%s, %s, %s, %s, %s)"

# Iterate through the JSON data and insert it into the table
for item in products_json: 
  product_data_tuple = (item["id"], item['name'], item["category"], item["subcategory"], item["image"])
  cursor.execute(insert_products_query, product_data_tuple)
  
# Commit the changes to the database
connection.commit()

In [4]:
# Stores Insertion

def get_field(data, field_names):
  for name in field_names:
    if name in data:
      return data[name]
  return "Supermarket"

def flatten_geometry(geometry):
  return geometry.get("coordinates", "")


# Read the JSON file
with open("./Data/stores.json", encoding="utf-8" ) as file:
  stores_json = json.load(file)

# create the insertion query
insert_stores_query = "INSERT INTO store (store_name, longitude, latitude) VALUES (%s, %s, %s)"

for item in stores_json:
  name_or_brand = get_field(item["properties"], ["name", "brand"])
  coordinates = item["geometry"].get("coordinates", "")
  latitude = coordinates[1]
  longitude = coordinates[0]
  store_data_tuple = (name_or_brand, longitude, latitude)
  cursor.execute(insert_stores_query, store_data_tuple)
  
# Commit the changes to the database
connection.commit()

In [5]:
# Categories Insertion

# Read the JSON file
with open("./Data/categories.json", encoding="utf-8" ) as file:
  categories_json = json.load(file)
  
# create the insertion query
insert_category_query = "INSERT INTO category (category_id, category_name) VALUES (%s, %s)"

# Iterate through the JSON data and insert it into the table
for item in categories_json:
  category_data_tuple = (item["id"], item["name"])
  cursor.execute(insert_category_query, category_data_tuple)
  
# Commit the changes to the database
connection.commit()

In [6]:
# Subcategories Insertion

# Read the JSON file
with open("./Data/categories.json", encoding="utf-8" ) as file:
  subcategories_json = json.load(file)
  
# create the insertion query
insert_subcategory_query = "INSERT INTO subcategory (subcategory_id, subcategory_name, parent_id) VALUES (%s, %s, %s)"

def flatten_subcategories(subcategories):
  return subcategories.get("name"), subcategories.get("uuid")

# Iterate through the JSON data and insert it into the table
for item in subcategories_json:
  parent_category = item["id"]
  subcategories_array = item["subcategories"]
  for subcategory in subcategories_array:
    subcategory_data_tuple = (subcategory["uuid"], subcategory["name"], parent_category)
    cursor.execute(insert_subcategory_query, subcategory_data_tuple)
     
# Commit the changes to the database
connection.commit()

In [5]:
# Products in Store Insertion
import random
import datetime

# Read the JSON file
with open("./Data/dbProducts.json", encoding="utf-8") as file:
  db_products_json = json.load(file);
  
# create the insertion query 
insert_query_productsDB = "INSERT INTO productsinstore(product_name, product_id, price, date_product, category, subcategory) VALUES (%s, %s, %s, %s, %s, %s)"

# create dates from 1st of August 2023
start_date = datetime.date(2023, 8, 1)
# until the current date
end_date = datetime.date.today()

# Iterate through the JSON data
for item in db_products_json:
  
  # get the necessary fields
  product_id = item["id"]
  product_name = item["label"]
  category = item["category"]
  subcategory = item["subcategory"]
  
  
  # create a set to store unique dates
  unique_dates = set()
  
  # create distinct dates
  while len(unique_dates) < 3:
    random_date = start_date + datetime.timedelta(days = random.randint(0, (end_date - start_date).days))
    unique_dates.add(random_date)
    
  # for every distinct date, execute an insertion query
  for random_date in unique_dates:
    # create a random price for the average price of the product
    product_price = random.uniform(0.8, 2.5)
    
    data_tuple = (product_name, product_id, product_price, random_date, category, subcategory)
    cursor.execute(insert_query_productsDB, data_tuple)
    
# Commit the changes to the database
connection.commit()
  
  

In [5]:
# Close the cursor and connection
cursor.close()
connection.close()